In [8]:
# required packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# naviagete to get route name and route
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
time.sleep(3)

body = driver.find_element(By.TAG_NAME,"body")
for i in range(3):
    body.send_keys(Keys.PAGE_DOWN)


page1 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route1 = [i.text for i in page1]
link1 = [i.get_attribute('href') for i in page1]
ksrtc1 = pd.DataFrame({'route':route1,'route_link':link1})

time.sleep(3)
driver.find_element(By.XPATH,"//div[contains(text(),'2')]").click()
page2 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route2 = [i.text for i in page2]
link2 = [i.get_attribute('href') for i in page2]
ksrtc2 = pd.DataFrame({'route':route2,'route_link':link2})

ksrtc = pd.concat([ksrtc1,ksrtc2], ignore_index=True)


ksrtc_url = ksrtc['route_link']

In [9]:
ksrtc

,route,route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
import pandas as pd
import numpy as np

def initialize_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def scroll_to_bottom(driver):
    """Scrolls to the bottom of the page to ensure all content is loaded."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolling = True
    while scrolling:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            scrolling = False
        else:
            last_height = new_height

def get_bus_data(urls):
    """Fetch bus data for each URL in the list."""
    all_bus_data = pd.DataFrame()
    driver = initialize_driver()

    for url in urls:
        try:
            driver.get(url)
            time.sleep(10)

            # Attempt to click the view buses button
            try:
                button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]'))
                )
                button.click()
            except TimeoutException:
                pass  # Continue if button not found or not clickable

            # Scroll to load all data
            scroll_to_bottom(driver)

            # Extract data
            bus_route = driver.find_element(By.CSS_SELECTOR, 'h1[class="D136_h1"]').text
            bus_name = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')]
            bus_type = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')]
            departure_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')]
            duration = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')]
            arrival_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')]
            star_rating = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')]
            price = [i.text for i in driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')]
            seats_available = [int(i.text.split()[0]) for i in driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]')]

            # Pad lists with NaN to match max length
            max_length = max(len(bus_name), len(bus_type), len(departure_time), len(duration),
                             len(arrival_time), len(star_rating), len(price), len(seats_available))

            def pad_list(lst, length):
                return lst + [np.nan] * (length - len(lst))

            bus_name = pad_list(bus_name, max_length)
            bus_type = pad_list(bus_type, max_length)
            departure_time = pad_list(departure_time, max_length)
            duration = pad_list(duration, max_length)
            arrival_time = pad_list(arrival_time, max_length)
            star_rating = pad_list(star_rating, max_length)
            price = pad_list(price, max_length)
            seats_available = pad_list(seats_available, max_length)

            # Create DataFrame for current URL
            ksrtc_bus = pd.DataFrame({
                'Route Name': [bus_route] * max_length,
                'Route Link': [url] * max_length,
                'Bus Name': bus_name,
                'Bus Type': bus_type,
                'Departure Time': departure_time,
                'Duration': duration,
                'Arrival Time': arrival_time,
                'Star Rating': star_rating,
                'Price': price,
                'Seats Available': seats_available
            })

            # Append to main DataFrame
            all_bus_data = pd.concat([all_bus_data, ksrtc_bus], ignore_index=True)

        except WebDriverException:
            print(f"Encountered an issue with URL: {url}. Moving to the next.")

    driver.quit()
    return all_bus_data

def process_bus_data(df):
    """Processes bus data by converting columns to numeric and filling NaNs."""
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df['Star Rating'] = pd.to_numeric(df['Star Rating'], errors='coerce')
    df['Seats Available'] = pd.to_numeric(df['Seats Available'], errors='coerce')

    df['Price'] = df['Price'].fillna(df['Price'].mean().round(0).astype(int))
    df['Star Rating'] = df['Star Rating'].fillna(df['Star Rating'].mean().round(0).astype(int))
    df['Seats Available'] = df['Seats Available'].fillna(df['Seats Available'].mean().round(0).astype(int))
    
    return df

# Replace `bsrtc['route_link']` with the actual list of URLs.
urls = ksrtc['route_link']
all_bus_data = get_bus_data(urls)
processed_bus_data = process_bus_data(all_bus_data)
processed_bus_data

,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,MMK Travels,A/C Seater / Sleeper (2+1),22:30,09h 00m,07:30,4.5,1299.0,11.0
1,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,NS Transports,VE A/C Sleeper (2+1),22:45,08h 45m,07:30,4.0,1699.0,16.0
2,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,08h 25m,06:10,4.2,1699.0,12.0
3,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,AdCHANCHALAS TOURS AND TRAVELS,A/C Semi Sleeper (2+2),19:45,10h 15m,06:00,3.8,1699.0,9.0
4,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,DLT BUS,A/C Sleeper (2+1),22:30,09h 00m,07:30,4.1,1440.0,16.0
...,...,...,...,...,...,...,...,...,...,...
554,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,07h 19m,02:29,3.0,750.0,25.0
555,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,3.1,700.0,25.0
556,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,Holy Maria Travels,A/C Semi Sleeper (2+2),18:20,07h 50m,02:10,5.0,700.0,25.0
557,Kottayam to Kozhikode Bus,https://www.redbus.in/bus-tickets/kottayam-to-...,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),18:15,07h 10m,01:25,1.0,1100.0,25.0


In [11]:
processed_bus_data.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [12]:
processed_bus_data.to_csv("ksrtc.csv", index = False)